# 1. KBO 타자 OPS 예측

## 1.1 문제 정의
### 1.1.1 경진대회 소개

![경진대회소개 이미지](img/1.png)
![경진대회 개요](img/2.png)

![경진대회 배경](img/3.png)

### 1.1.2 평가척도

WRMSE(Weighted Root Mean Squeare Error)
![경진대회 배경](img/4.png)
실제값과 예측값의 차이를 기준으로 평가하므로 오차가 적을수록 좋음.

타석에 들어서지 못한 선수들은 가중치가 0이므로 계산 결과도 0이됨.

![WRMSE 평가 예시](img/5.png)

### 1.1.3 도메인 조사
![분석에 필요한 주요 야구 용어](img/6.png)

## 1.2. 탐색적 데이터 분석

In [ ]:
# 필요 라이브러리 로드



# 윈도우인 경우 맑은 고딕 폰트 이용
# Mac 인 경우


#그래프에서 마이너스 기호가 표시되게 하는 설정입니다.


### 1.2.1. 프리시즌 데이터 분석

In [ ]:
# 프리시즌 데이터 로드

# 정규시즌 데이터 로드

# 데이터 크기 확인

# 데이터 상단 출력


In [ ]:
# 데이터 기초통계량 확인


In [ ]:
# 데이터 시각화


In [ ]:
# 정규시즌 데이터에서 2002년 이후의 연도별 기록된 선수의 수


In [ ]:
# 프리시즌 데이터에서 연도별 기록된 선수의 수


In [ ]:
#프리시즌 성적과 정규시즌의 성적 간의 상관관계를 살펴보기 위해 두 데이터에 공통적으로 존제하는 선수들의 기록만 정리

# 타자의 이름과 연도를 이용해 새로운 인덱스를 생성


In [ ]:
# 새로운 인덱스의 교집합


In [ ]:
# 교집합에 존재하는 데이터만 불러오기


In [ ]:
# 비교를 위해 인덱스로 정렬


In [ ]:
# 검정 코드


In [ ]:
# 정규시즌과 프리시즌의 상관관계 계산


### 1.2.2. 정규시즌 데이터 분석

<a id='m1'>연도별 OPS 중앙값 그래프</a>

<a id='m2'>연도별 팀 OPS</a>

In [ ]:
# 연도별 팀의 OPS 중앙값 계산


In [ ]:
# 2005년 이후에 결측치가 존재하지 않는 팀만 확인


In [ ]:
# 몸무게/키 계산


# 정규시즌과 프리시즌의 상관관계 계산


# 정규시즌과 프리시즌의 상관관계 계산



In [ ]:
# position


# 우타, 좌타, 양타


In [ ]:
# career를 split

# 외국만 추출

# 결측치 처리


In [ ]:
# 결측치라면 그대로 0으로 두고 ‘만원’이 포함되어 있다면 숫자만 뽑아서 초봉으로 넣어준다. 그외 만 원 단위가 아닌 초봉은 결측치로 처리한다.


### 1.2.3. 일별 데이터 분석

In [ ]:
# 날짜(date)를 ‘.’을 기준으로 나누고 첫 번째 값을 월(month)로 지정 


# 각 연도의 월별 평균 누적 타율(avg2) 계산


# pivot_table을 이용해 데이터 변형


<a id='m3'>연도별 월 평균 타율</a>

## 1.3. 데이터 전처리

In [ ]:
# 수치형 타입의 변수 저장


# 수치형 타입 변수 중 결측치가 하나라도 존재하는 행 출력
# isna().sum(axis=1) -> 열 기준의 결측치 개수
# df.loc[]를 통해 결측치 0개 이상 데이터를 추출


In [ ]:
# 수치형 변수에 포함되는 데이터 타입 선정


# 정규 시즌 데이터에서 결측치를 0으로 채우기


In [ ]:
# 일별 데이터에서 결측치를 0으로 채우기


In [ ]:
# 프리시즌 데이터에서 결측치를 0으로 채우기


In [ ]:
# 수치형이 아닌 변수 추출


# 수치형이 아닌 변수 중 결측치가 하나라도 존재하는 행 출력
# isna().sum(axis=1) -> 열 기준의 결측치 개수
# df.loc[]를 통해 결측치 0개 이상 데이터를 추출


In [ ]:
# 삭제할 데이터 추출
        

# 데이터 삭제


### 1.3.2. 규정 타수 정의

In [ ]:
# OPS 이상치 탐색을 위한 수치 정의


# 실제 OPS 이상치 탐색


<a id='m4'>7월 일별 경기수 합</a>

In [ ]:
# 7.01~7.31 숫자 생성 후 반 올림


### 1.3.3. 시간 변수

In [ ]:
# 시간 변수를 생성하는 함수 정의


In [ ]:
# 상관관계를 탐색할 변수 선택


In [ ]:
# 시간변수 생성 함수를 통한 지표별 1년 전 성적 추출


In [ ]:
# 상관관계 도출


In [ ]:
# 상관관계의 시각적 표현


# 대각 행렬을 기준으로 한 쪽만 나타나게 설정해줍니다.



In [ ]:
#희생 플라이 구하기
#OBP(출루율) 계산 공식 이용하여 SF(희생 플라이)계산 >> (H+BB+HBP)/OBP-(AB+BB+HBP)


#한 타수당 평균 희생 플라이 계산 후 필요한 것만 추출


In [ ]:
#day_by_day에서 연도별 선수의 시즌 전반기 출루율과 관련된 성적 합 구하기

#day_by_day와 regular season에서 구한 희생 플라이 관련 데이터를 합치기

#선수별 전반기 희생 플라이 수 계산

#선수별 전반기 OBP(출루율) 계산

# OBP 결측치를 0으로 처리 

# 분석에 필요하지 않은 열 제거


### 1.3.4. 추가 변수 생성

<a id='m5'>나이별 평균 성적</a>

In [ ]:
# 나이 변수 생성


In [ ]:
# 나이, 평균 출루율, 출루율 중위값으로 구성된 데이터프레임 구축


In [ ]:
# 나이에 따른 출루율 추이 시각화


In [ ]:
# 나이를 포함한 변수 선택


# 총 3년 전 성적까지 변수를 생성


### 1.3.5. 데이터 사후 처리

In [ ]:
# 결측치 비율 확인


In [ ]:
#1. 선수별 OBP 평균
# SF = (H+BB+HBP) / OBP-(AB+BB+HBP)
# OBP = (H+BB+HBP) / (AB+BB+HBP+SF)


In [ ]:
#2. 시즌별 OBP 평균


In [ ]:
#### player_OBP_mean(선수평균) 열 추가


In [ ]:
# 결측치 처리하는 함수 정의


In [ ]:
# 생성한 함수를 이용해 결측치 처리 진행


### 1.3.6. SLG 데이터 전처리

In [ ]:
# 상관관계를 탐색할 변수 선택


# 시간변수 생성 함수를 통한 지표별 1년 전 성적 추출

# 상관관계 도출


# 상관관계의 시각적 표현


# 대각 행렬을 기준으로 한쪽만 나타나게 설정해줍니다.



In [ ]:
# day_by_day에서 연도별 선수의 시즌 전반기 장타율(SLG)과 관련된 성적 합 구하기

# 전반기 장타율 계산

# SLG 결측치를 0으로 처리 

# 필요한 칼럼만 불러오고 나이 계산


In [ ]:
# 총 3년 전 성적까지 변수를 생성


In [ ]:
# 선수별 SLG 평균 데이터(player_SLG_mean)를 만듭니다


# 시즌별 SLG 평균 데이터(season_SLG_mean)를 만듭니다


# 선수 평균의 SLG(player_OBP_mean)를 새로운 변수로 더합니다.


# 선수 평균의 성적이 결측치이면 데이터에서 제거합니다.


# 결측치 처리



## 1.4. 모델링
### 1.4.1. 데이터 분할

In [ ]:
# 30타수 이상의 데이터만 학습


# 2018년 데이터를 test 데이터 2018년 이전은 train 데이터로 나눈다.




### 1.4.2. 모델 선택

In [ ]:


# log 단위(1e+01)로 1.e-04 ~ 1.e+01 사이의 구간에 대해 parameter를 탐색한다. 


# GridSearchCV를 이용하여 dict에 Lasso, Ridege OBP 모델을 저장한다.


# GridSearchCV를 이용하여 dict에 Lasso, Ridge SLG 모델을 저장한다


In [ ]:


# 랜덤 포레스트의 parameter 범위를 정의한다.


# GridSearchCV를 이용하여 dict에 OBP Randomforest 모델을 저장한다.


# GridSearchCV를 이용하여 dict에 OBP Randomforest 모델을 저장한다.



In [ ]:
# 테스트 데이터셋(2018년)의 선수들의 OBP를 예측

# XGB_OBP = XGB_OBP_gridsearch.predict(OBP_test.iloc[:,-5:])

# test 데이터의 WRMSE 계산

# 막대그래프 위에 값을 표시해준다.


In [ ]:
# 테스트 데이터셋(2018년)의 선수들의 SLG를 예측


# test데이터 WRMSE 계산



### 1.4.3. 결과 해석과 평가

In [ ]:
# Lasso에서 GridSearchCV로 탐색한 최적의 alpha값 출력


## 1.5. 성능 향상을 위한 방법
### 1.5.1. 앙상블

### 1.5.2. 단순화된 모델 생성

### 1.5.3. 테스트 데이터 정제

### 1.5.4. 반발계수의 변화